Library Imports

In [1]:
import csv
import pandas as pd

In [ ]:
#code for reading file
def readf(file):
    f = open(file, "r")
    data = list(csv.reader(f))
    f.close()
    return data



['1715911', '14302', '42369398', 'guru', '', 'PscInspectionId: 1715911\n\nDeficiency/Finding: Engine room sea chest valve open/close marking not in contrasting colour.\n\nDescription Overview: UNKNOWN\n\nImmediate Causes: All sea chest valves are periodically checked, maintained and records are maintained. During the inspection , The sea chest valve open/close marking is not marked with a contrasting color, making it difficult to confirm their identification during visual inspections, resulting in the observation. The crew members was aware of the inspection requirements for the onboard sea chest valves, and carried out monthly inspections.\n\nRoot Cause Analysis: Incorrect Marking\n\nCorrective Action: 1. All onboard valves with open/closing markings are inspected and marked with contrasting colors for easy identification. 2. Briefed on board by chief engineer and emphasized on the importance of marking.\n\nPreventive Action: Vessel superintendent discussed the observation with Master

In [ ]:

scores = {}
count = {}
convertion = {"Low": 1, }
data = readf("psc_severity_train.csv")
print(data[5014])
for row in data:
    if row[4] != '':
        if row[0] not in scores:
            count[row[0]] = 1
            
        else:
            count[row[0]] += 1